# Markowitz Portfolio Optimization

## 1. Problem Formulation
We aim to construct an **optimal portfolio** that balances **risk** and **return** using **Markowitz Portfolio Theory**. The optimization problem is formulated as a **quadratic programming (QP) model**, solved using the **Gurobi optimizer**.

### 1.1 Decision Variables
Let:
- $ w_i $ be the weight (fraction of total capital) allocated to asset $ i $.
- $ n $ be the total number of assets.

### 1.2 Objective Function
We consider two possible objectives:

#### A. **Minimizing Portfolio Risk (Variance)**
$$ \min \sum_{i=1}^{n} \sum_{j=1}^{n} w_i w_j \sigma_{ij} $$
where:
- $ \sigma_{ij} $ is the covariance between asset $ i $ and asset $ j $.

#### B. **Maximizing Expected Portfolio Return**
$$ \max \sum_{i=1}^{n} w_i \mu_i $$
where:
- $ \mu_i $ is the expected return of asset $ i $.

### 1.3 Constraints
1. **Fully Invested Portfolio**: The sum of asset weights must equal 1.
   $$ \sum_{i=1}^{n} w_i = 1 $$

2. **Minimum Expected Return Constraint**: The portfolio return must be at least a target return $ R_t $.
   $$ \sum_{i=1}^{n} w_i \mu_i \geq R_t $$

3. **Risk Constraint**: The portfolio risk (variance) must be at most a target risk $ V_t $.
   $$ \sum_{i=1}^{n} \sum_{j=1}^{n} w_i w_j \sigma_{ij} \leq V_t $$

4. **Non-Negativity of Weights**: No short-selling is allowed.
   $$ w_i \geq 0, \quad \forall i \in \{1, 2, ..., n\} $$

## 2. Efficient Frontier
The **Efficient Frontier** represents portfolios that achieve the highest return for a given level of risk. We generate this by:
1. **Varying the target return $ R_t $** across a range.
2. **Solving the optimization problem** for each $ R_t $ to find the minimum risk portfolio.
3. **Plotting risk vs. return** to visualize the efficient frontier.

### Note:
The above model shows both of the possible objective functions and all the constraints. However, in practice, you would typically choose to minimize the portfolio risk for a given target return.

## Code:

In [ ]:
import yfinance as yf
import gurobipy as gb
from gurobipy import GRB
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# ------------------------------
# Data Collection & Preprocessing
# ------------------------------
def fetch_stock_data(tickers, start_date, end_date):
    """Fetch historical adjusted closing prices for given stock tickers."""
    all_data = []
    for ticker in tickers:
        try:
            data = yf.download(ticker, start=start_date, end=end_date, progress=False)["Adj Close"]
            if not data.empty:
                all_data.append(data.rename(ticker))
        except Exception as e:
            print(f"Error fetching {ticker}: {e}")
    
    if all_data:
        stock_data = pd.concat(all_data, axis=1)
        stock_data.fillna(method="ffill", inplace=True)  # Handle missing data
        return stock_data
    else:
        raise ValueError("No valid stock data retrieved.")

def compute_returns_and_cov(stock_data):
    """Computes daily returns and covariance matrix."""
    returns = stock_data.pct_change().dropna()
    return returns.mean(), returns.cov()

In [ ]:
# ------------------------------
# Portfolio Optimization using Gurobi
# ------------------------------
def optimize_portfolio(mean_returns, cov_matrix, target_return=None):
    """Solves the Markowitz portfolio optimization problem using Gurobi."""
    num_assets = len(mean_returns)
    model = gb.Model("Markowitz Portfolio Optimization")
    model.Params.OutputFlag = 0  # Suppress solver output
    
    # Decision variables (weights)
    weights = model.addVars(num_assets, lb=0.0, ub=1.0, name="Weights")
    
    # Constraints
    model.addConstr(gb.quicksum(weights[i] for i in range(num_assets)) == 1, name="Budget")
    
    if target_return is not None:
        model.addConstr(gb.quicksum(weights[i] * mean_returns[i] for i in range(num_assets)) >= target_return, name="TargetReturn")
    
    # Objective: Minimize portfolio variance
    portfolio_variance = gb.quicksum(weights[i] * weights[j] * cov_matrix.iloc[i, j] for i in range(num_assets) for j in range(num_assets))
    model.setObjective(portfolio_variance, GRB.MINIMIZE)
    
    # Solve optimization problem
    model.optimize()
    
    if model.status == GRB.Status.OPTIMAL:
        return {
            "weights": np.array([weights[i].x for i in range(num_assets)]),
            "variance": model.ObjVal,
            "std_dev": np.sqrt(model.ObjVal)
        }
    else:
        raise ValueError("Optimization failed: No optimal solution found.")

In [ ]:
# ------------------------------
# Efficient Frontier Calculation
# ------------------------------
def compute_efficient_frontier(mean_returns, cov_matrix, num_portfolios=50):
    """Generates the efficient frontier by solving optimization for different target returns."""
    min_return, max_return = mean_returns.min(), mean_returns.max()
    target_returns = np.linspace(min_return, max_return, num_portfolios)
    
    port_risks, port_weights = [], []
    
    for target_return in target_returns:
        try:
            result = optimize_portfolio(mean_returns, cov_matrix, target_return)
            port_risks.append(result["std_dev"])
            port_weights.append(result["weights"])
        except:
            continue  # Skip infeasible points
    
    return target_returns[:len(port_risks)], port_risks, port_weights

In [ ]:
# ------------------------------
# Visualization of the Efficient Frontier
# ------------------------------
def plot_efficient_frontier(target_returns, port_risks):
    """Plots the efficient frontier of the portfolio optimization."""
    plt.figure(figsize=(12, 6))
    plt.scatter(port_risks, target_returns, c=target_returns / np.array(port_risks), marker="o", cmap="viridis")
    plt.colorbar(label="Sharpe Ratio")
    plt.xlabel("Risk (Standard Deviation)")
    plt.ylabel("Expected Return")
    plt.title("Efficient Frontier")
    plt.show()

In [ ]:
# ------------------------------
# Main Execution
# ------------------------------
if __name__ == "__main__":
    tickers = ["AAPL", "GOOGL", "AMZN", "NFLX", "TSLA"] # Select the stock tickers you desire
    start_date, end_date = "2020-01-01", "2024-11-10" # Select the range of the historical data you desire
    
    print("Fetching stock data...")
    stock_data = fetch_stock_data(tickers, start_date, end_date)
    mean_returns, cov_matrix = compute_returns_and_cov(stock_data)
    
    print("Computing efficient frontier...")
    target_returns, port_risks, _ = compute_efficient_frontier(mean_returns, cov_matrix)
    
    print("Plotting efficient frontier...")
    plot_efficient_frontier(target_returns, port_risks)